In [24]:
import pandas as pd
import numpy as np
import random
from sklearn.base import clone

In [25]:
class BaggingClassifier:
    def __init__(self, model, n_models=10, elements_per_sample=100):
        self.model = model
        self.n_models = n_models
        self.elements_per_sample = elements_per_sample
        self.models = []
    def fit(self, X_train, y_train):
        df = pd.concat([X_train,y_train], axis=1)
        for i in range(self.n_models):
            train_set = df.sample(self.elements_per_sample, random_state=42)
            curr_model = clone(self.model)
            curr_model.fit(train_set.iloc[:,:-1], train_set.iloc[:,-1])
            self.models.append(curr_model)
    def predict(self,X_test):
        predictions = []
        for model in self.models:
            pr = model.predict(X_test)
            predictions.append(pr.reshape(-1,1))
        return np.round(np.mean(np.concatenate(predictions, axis=1), axis=1)).astype(int)

In [26]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [27]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3, metric='minkowski', p=1)
knn.fit(X_train, y_train)
y_knn_pred = knn.predict(X_test)

In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_knn_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.85      0.94      0.89        18
           2       0.96      0.89      0.92        27

    accuracy                           0.93        60
   macro avg       0.94      0.94      0.94        60
weighted avg       0.94      0.93      0.93        60



In [33]:
bag = BaggingClassifier(KNeighborsClassifier(n_neighbors=3, metric='minkowski', p=1), 10, 40)
bag.fit(pd.DataFrame(X_train), pd.DataFrame(y_train))
bag_pred = bag.predict(X_test)

In [34]:
print(classification_report(y_test, bag_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.94      0.94      0.94        18
           2       0.96      0.96      0.96        27

    accuracy                           0.97        60
   macro avg       0.97      0.97      0.97        60
weighted avg       0.97      0.97      0.97        60

